In [5]:
from itertools import combinations

def unify(x, y, theta={}):
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    else:
        return None

def unify_var(var, x, theta):
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def resolve(clause1, clause2):
    for predicate1 in clause1:
        for predicate2 in clause2:
            theta = unify(predicate1, negate(predicate2))
            if theta is not None:
                new_clause = (substitute(clause1, theta) | substitute(clause2, theta)) - {predicate1, predicate2}
                return frozenset(new_clause)
    return None

def negate(predicate):
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def substitute(clause, theta):
    return {substitute_predicate(p, theta) for p in clause}

def substitute_predicate(predicate, theta):
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])

def resolution(kb, query):
    negated_query = frozenset({negate(query)})
    clauses = kb | {negated_query}
    new_clauses = set()
    resolution_steps = []

    while True:
        pairs = list(combinations(clauses, 2))
        for clause1, clause2 in pairs:
            resolvent = resolve(clause1, clause2)
            if resolvent is not None:
                resolution_steps.append((clause1, clause2, resolvent))
                if frozenset() == resolvent:  # Empty clause found
                    print_resolution_path(resolution_steps)
                    return True
                new_clauses.add(resolvent)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

def print_resolution_path(resolution_steps):
    print("\nFinal Resolution Path:")
    for i, (c1, c2, resolvent) in enumerate(resolution_steps[-3:], 1):  # Show only the last few steps
        print(f"Step {i}: {c1} + {c2} => {resolvent}")
    print("\nEmpty clause derived. Query proven!")

# Knowledge Base (in CNF)
kb = {
    frozenset({('Mother', 'leela', 'Oshin')}),
    frozenset({('Alive', 'leela')}),
    frozenset({('not', ('Mother', 'x', 'y')), ('Parent', 'x', 'y')}),
    frozenset({('not', ('Parent', 'x', 'y')), ('not', ('Alive', 'x')), ('Older', 'x', 'y')}),
}

# Query: Prove that Leela is older than Oshin
query = ('Older', 'leela', 'Oshin')

print("\nADITYA RAM S H\n1BM22CS019\n")
result = resolution(kb, query)
if result:
    print("\nProven: Leela is older than Oshin.")
else:
    print("\nCannot prove: Leela is older than Oshin.")



ADITYA RAM S H
1BM22CS019


Final Resolution Path:
Step 1: frozenset({('not', ('Parent', 'x', 'y'))}) + frozenset({('Alive', ('not', ('Parent', 'x', 'y'))), ('Older', 'x', 'y'), ('not', ('Alive', 'x'))}) => frozenset({('not', ('Alive', 'x')), ('Older', ('Parent', 'x', 'y'), 'y')})
Step 2: frozenset({('not', ('Parent', 'x', 'y'))}) + frozenset({('not', ('Alive', 'x')), ('Older', ('Parent', 'x', 'y'), 'y'), ('Mother', ('not', ('Parent', 'x', 'y')), 'Oshin')}) => frozenset({('Older', ('Parent', 'x', 'y'), 'y'), ('Mother', ('not', ('Parent', 'x', 'y')), 'Oshin')})
Step 3: frozenset({('not', ('Parent', 'x', 'y'))}) + frozenset({('not', ('Parent', 'x', 'y')), ('Mother', ('not', ('Parent', 'x', 'y')), 'Oshin')}) => frozenset()

Empty clause derived. Query proven!

Proven: Leela is older than Oshin.
